# CS572 Project
* We selected the paper: {}
* Based on the data and model of given paper, we are planning to make an extension of LSTM that predicts {}.

## Preparing

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import time
from matplotlib import pyplot as plt

In [2]:
input_size = 8
hidden_size = 128
num_layers = 3
num_epochs = 2000
batch_size = 32
learning_rate = 0.001
weight_decay=0.00001

Check if your computer is prepared to run pytorch model with CUDA.

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # x -> batch_size, seq, input_size
        self.fc = nn.Linear(hidden_size, 1)
        
    def makehc_zero(self, batch_size):
        self.hn = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        self.cn = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
#         print("clear")
        
    
    def forward(self, x):
        if self.training == True:
            # Set initial hidden and cell states 
            self.makehc_zero(x.size(0))
#         out, _ = self.rnn(x, h0)
        
        # Forward propagate LSTM
        out, (self.hn, self.cn) = self.lstm(x, (self.hn, self.cn))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        
        # Decode the hidden state of the last time step
        # out : batch_size, seq_length, hidden_size
        # out (N, 9, 128)  ???
        
        out_seq = [self.fc(out[:,i,:]) for i in range(out.size(1))]
        out = torch.cat(out_seq, dim=1)
#         print(out.shape)
        
        return out

Data Loading

In [5]:
training_set = []
data_path = 'Data\Training' # add path

for path_idx in os.listdir(data_path):
    cur_path = os.path.join(data_path, path_idx)
    print(cur_path)
    input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
    # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
    inputs = torch.Tensor(input_file)
#     print(inputs[0])
    
    inputs = inputs[:len(inputs)-len(inputs)%40]
#     labels = inputs[:, [5]]
# #     print(labels.shape)
#     labels = labels.view([-1, 40, 1])
    
    inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
    inputs = inputs.view([-1,40,9])
    
#     for test a file
    training_set += inputs

testing_set = []
data_path = 'Data\Testing' # add path
for path_idx in os.listdir(data_path):
    cur_path = os.path.join(data_path, path_idx)
    print(cur_path)
    input_file = np.loadtxt(cur_path, dtype='float', delimiter=',')
    # cell = nn.RNN(input_size=4, hidden_size=2, batchfirst=True)
    inputs = torch.Tensor(input_file)
#     print(inputs[0])
    
    inputs = inputs[:len(inputs)-len(inputs)%40]
#     labels = inputs[:, [5]]
# #     print(labels.shape)
#     labels = labels.view([-1, 40, 1])
    
    inputs = inputs[:, [1,2,3,4,6,7,8,9,5]]
    inputs = inputs.view([-1,40,9])
    
#     for test a file
    testing_set += inputs
   
# print(data_set)
print("load complete")

Data\Training\190722_tension_test_space_1_trial_2.csv
Data\Training\190722_tension_test_space_1_trial_3.csv
Data\Training\190722_tension_test_space_2_trial_1.csv
Data\Training\190722_tension_test_space_2_trial_2.csv
Data\Training\190722_tension_test_space_2_trial_4.csv
Data\Training\190722_tension_test_space_3_trial_2.csv
Data\Training\190722_tension_test_space_3_trial_3.csv
Data\Training\190722_tension_test_space_3_trial_4.csv
Data\Training\190722_tension_test_space_3_trial_5.csv
Data\Training\190722_tension_test_space_3_trial_6.csv
Data\Training\190722_tension_test_space_4_trial_2.csv
Data\Training\190722_tension_test_space_4_trial_3.csv
Data\Training\190722_tension_test_space_5_trial_2.csv
Data\Training\190722_tension_test_space_5_trial_3.csv
Data\Training\test11thick_1.csv
Data\Training\test11thick_3.csv
Data\Training\test22thick_2.csv
Data\Training\test33thick_2.csv
Data\Training\test33thick_3.csv
Data\Training\test55thick_2.csv
Data\Training\test55thick_3.csv
Data\Testing\190722_

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=training_set,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testing_set,
                                          batch_size=1, 
                                          shuffle=False)
print("loader setting complete")

loader setting complete


In [7]:
# Training and Testing: To train fingertip forces from input
# data, a three-layer BT-LSTM with 128 hidden units is used.
# Sequence time length of BT-LSTM was set to T = 40. The final
# output of the BT-LSTM layer is fed into the final fully-connected
# layer to obtain the one-dimensional output. The mean squared
# error (MSE) function is used to measure the loss between the
# predicted and the ground truth contact force. During training,
# the number of epoch was 2000, and the mini batch size was 32.
# Adam optimizer was used with 0.001 learning rate and 10−5
# weight decay
lstm = LSTM(input_size, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=weight_decay)

Load cell if needed

In [ ]:
lstm.load_state_dict(torch.load('LSTM.ckpt'))

Train model

In [ ]:
for i, x in enumerate(train_loader):
#     print (i, x)
#     print(x.shape)
#     inputs, labels = torch.split(x, [8, 1], 2)
#     print(inputs.shape)
#     print(labels.shape)
    break

In [ ]:
for i in test_loader:
    print (i)
    break

In [ ]:
# Train the model
start = time.time()
print("training start")
for epoch in range(num_epochs):
# for epoch in range(200):    
    for i, inputs in enumerate(train_loader):
        inputs, labels = torch.split(inputs, [8, 1], 2)
        inputs = inputs.to(device)
        labels = labels.squeeze().to(device)
        
        # Forward pass
        outputs = lstm(inputs)
        
#         print(labels.shape)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, loss.item()))
        print('elapsed time :', time.time() - start)
print("training complete")
print('total_training elapsed time :', time.time() - start)

training start
Epoch [10/2000], Loss: 3.5906
elapsed time : 72.07684373855591
Epoch [20/2000], Loss: 1.5470
elapsed time : 144.9637417793274
Epoch [30/2000], Loss: 1.5745
elapsed time : 217.28625869750977
Epoch [40/2000], Loss: 2.2112
elapsed time : 290.08675932884216
Epoch [50/2000], Loss: 0.4893
elapsed time : 363.2852544784546
Epoch [60/2000], Loss: 2.1556
elapsed time : 435.5586516857147
Epoch [70/2000], Loss: 0.9105
elapsed time : 508.458904504776
Epoch [80/2000], Loss: 2.8663
elapsed time : 580.8677232265472
Epoch [90/2000], Loss: 5.0020
elapsed time : 653.1193611621857
Epoch [100/2000], Loss: 0.1652
elapsed time : 724.5843496322632
Epoch [110/2000], Loss: 0.8182
elapsed time : 796.7147650718689
Epoch [120/2000], Loss: 0.6453
elapsed time : 868.1304385662079
Epoch [130/2000], Loss: 1.3649
elapsed time : 939.9476895332336
Epoch [140/2000], Loss: 1.1188
elapsed time : 1011.8189153671265
Epoch [150/2000], Loss: 2.1779
elapsed time : 1084.1855487823486
Epoch [160/2000], Loss: 1.2901


Test model

In [ ]:
lstm.eval()
start = time.time()
with torch.no_grad():
    total_labels = []
    total_outputs = []
    lstm.makehc_zero(1) #test batch size = 1
    for inputs in test_loader:
        inputs, labels = torch.split(inputs, [8, 1], 2)
        
        
        inputs = inputs.to(device)
        labels = labels.squeeze()
        
        for i in range(inputs.size(1)):
            outputs = lstm(inputs[:,i,:].view([inputs.size(0),1,8]))
            total_outputs += outputs.cpu()
        
        total_labels += labels
    rmse = mean_squared_error(total_labels, total_outputs, squared = False)
    mae = mean_absolute_error(total_labels, total_outputs)

    print('Test Accuracy of the model RMSE: {}'.format(rmse))
    print('Test Accuracy of the model MAE: {}'.format(mae)) 
    print('testing elapsed time :', time.time() - start)

Save model

In [ ]:
torch.save(lstm.state_dict(), 'LSTM_2000epoch.ckpt')
print("save complete")

Plot model

In [ ]:
plt.plot(total_outputs)
plt.plot(total_labels)
plt.show()

In [ ]:
plt.plot(total_outputs)
plt.plot(total_labels)
plt.savefig('LSTM_compare_all_labels_2000epoch.png')

In [ ]:
print(total_labels)